<a href="https://colab.research.google.com/github/JonasZimmer1994/deeplearning_wavenet/blob/master/wavenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl 

     |████████████████████████████████| 592.3MB 1.1MB/s 
ERROR: torchvision 0.4.2+cu100 has requirement torch==1.3.1, but you'll have torch 0.3.0.post4 which is incompatible.
ERROR: fastai 1.0.59 has requirement torch>=1.0.0, but you'll have torch 0.3.0.post4 which is incompatible.
  Found existing installation: torch 1.3.1
    Uninstalling torch-1.3.1:
      Successfully uninstalled torch-1.3.1


In [0]:
!pip3 install http://download.pytorch.org/whl/cu92/torch-0.4.1-cp36-cp36m-linux_x86_64.whl
!pip3 install torchvision

In [0]:
!git clone https://github.com/vincentherrmann/pytorch-wavenet.git

In [5]:
%cd /content/pytorch-wavenet/

/content/pytorch-wavenet


In [6]:
import torch
print('Torch', torch.__version__, 'CUDA', torch.version.cuda)


from wavenet_model import *
from audio_data import WavenetDataset
from wavenet_training import *
from model_logging import *

Torch 0.3.0.post4 CUDA 8.0.61


In [0]:
# initialize cuda option
dtype = torch.FloatTensor # data type
ltype = torch.LongTensor # label type

#use_cuda = torch.cuda.is_available()
#if use_cuda:
#    print('use gpu')
#    dtype = torch.cuda.FloatTensor
#    ltype = torch.cuda.LongTensor

In [8]:
model = WaveNetModel(layers=5,
                     blocks=3,
                     dilation_channels=32,
                     residual_channels=32,
                     skip_channels=1024,
                     end_channels=512, 
                     output_length=16,
                     dtype=dtype, 
                     bias=True)
# model = load_latest_model_from('snapshots', use_cuda=use_cuda)

print('model: ', model)
print('receptive field: ', model.receptive_field)
print('parameter count: ', model.parameter_count())



model:  WaveNetModel(
  (filter_convs): ModuleList(
    (0): Conv1d (32, 32, kernel_size=(2,), stride=(1,))
    (1): Conv1d (32, 32, kernel_size=(2,), stride=(1,))
    (2): Conv1d (32, 32, kernel_size=(2,), stride=(1,))
    (3): Conv1d (32, 32, kernel_size=(2,), stride=(1,))
    (4): Conv1d (32, 32, kernel_size=(2,), stride=(1,))
    (5): Conv1d (32, 32, kernel_size=(2,), stride=(1,))
    (6): Conv1d (32, 32, kernel_size=(2,), stride=(1,))
    (7): Conv1d (32, 32, kernel_size=(2,), stride=(1,))
    (8): Conv1d (32, 32, kernel_size=(2,), stride=(1,))
    (9): Conv1d (32, 32, kernel_size=(2,), stride=(1,))
    (10): Conv1d (32, 32, kernel_size=(2,), stride=(1,))
    (11): Conv1d (32, 32, kernel_size=(2,), stride=(1,))
    (12): Conv1d (32, 32, kernel_size=(2,), stride=(1,))
    (13): Conv1d (32, 32, kernel_size=(2,), stride=(1,))
    (14): Conv1d (32, 32, kernel_size=(2,), stride=(1,))
  )
  (gate_convs): ModuleList(
    (0): Conv1d (32, 32, kernel_size=(2,), stride=(1,))
    (1): Conv1d

In [9]:
data = WavenetDataset(dataset_file='train_samples/bach_chaconne/dataset.npz',
                      item_length=model.receptive_field + model.output_length - 1,
                      target_length=model.output_length,
                      file_location='train_samples/bach_chaconne',
                      test_stride=500)
print('the dataset has ' + str(len(data)) + ' items')

one hot input
the dataset has 47899 items


In [0]:
def generate_and_log_samples(step):
    sample_length=32000
    gen_model = load_latest_model_from('snapshots', use_cuda=False)
    print("start generating...")
    samples = generate_audio(gen_model,
                             length=sample_length,
                             temperatures=[0.5])
    tf_samples = tf.convert_to_tensor(samples, dtype=tf.float32)
    logger.audio_summary('temperature_0.5', tf_samples, step, sr=16000)

    samples = generate_audio(gen_model,
                             length=sample_length,
                             temperatures=[1.])
    tf_samples = tf.convert_to_tensor(samples, dtype=tf.float32)
    logger.audio_summary('temperature_1.0', tf_samples, step, sr=16000)
    print("audio clips generated")


logger = TensorboardLogger(log_interval=200,
                           validation_interval=400,
                           generate_interval=1000,
                           generate_function=generate_and_log_samples,
                           log_dir="logs/chaconne_model")

# logger = Logger(log_interval=200,
#                 validation_interval=400,
#                 generate_interval=1000)



In [11]:
trainer = WavenetTrainer(model=model,
                         dataset=data,
                         lr=0.001,
                         snapshot_path='snapshots',
                         snapshot_name='chaconne_model',
                         snapshot_interval=1000,
                         logger=logger,
                         dtype=dtype,
                         ltype=ltype)


print('start training...')
trainer.train(batch_size=16,
              epochs=3)

start training...
epoch 0
one training step does take approximately 0.36115028858184817 seconds)

load model snapshots/chaconne_model_2019-11-14_17-25-31
start generating...
one generating step does take approximately 0.012534172534942628 seconds)
one generating step does take approximately 0.015022222995758058 seconds)
Last generate is still running, skipping this one
audio clips generated
epoch 1
load model snapshots/chaconne_model_2019-11-14_17-48-41
start generating...
one generating step does take approximately 0.020611040592193604 seconds)
one generating step does take approximately 0.020516624450683595 seconds)
Last generate is still running, skipping this one
audio clips generated
load model snapshots/chaconne_model_2019-11-14_18-13-01
start generating...
one generating step does take approximately 0.01784022808074951 seconds)
one generating step does take approximately 0.01822490930557251 seconds)
epoch 2
Last generate is still running, skipping this one
audio clips generated


In [13]:
start_data = data[25000][0] # use start data from the data set
start_data = torch.max(start_data, 0)[1] # convert one hot vectors to integers

def prog_callback(step, total_steps):
    print(str(100 * step // total_steps) + "% generated")


generated = model.generate_fast(num_samples=160000,
                                 first_samples=start_data,
                                 progress_callback=prog_callback,
                                 progress_interval=1000,
                                 temperature=1.0,
                                 regularize=0.)

0% generated
one generating step does take approximately 0.004016220569610596 seconds)
0% generated
1% generated
1% generated
2% generated
3% generated
3% generated
4% generated
4% generated
5% generated
6% generated
6% generated
7% generated
8% generated
8% generated
9% generated
9% generated
10% generated
11% generated
11% generated
12% generated
13% generated
13% generated
14% generated
14% generated
15% generated
16% generated
16% generated
17% generated
18% generated
18% generated
19% generated
19% generated
20% generated
21% generated
21% generated
22% generated
23% generated
23% generated
24% generated
24% generated
25% generated
26% generated
26% generated
27% generated
28% generated
28% generated
29% generated
29% generated
30% generated
31% generated
31% generated
32% generated
33% generated
33% generated
34% generated
34% generated
35% generated
36% generated
36% generated
37% generated
38% generated
38% generated
39% generated
39% generated
40% generated
41% generated
41% g

In [14]:
import IPython.display as ipd

ipd.Audio(generated, rate=16000)